In [1]:
#if run locally or colab
import os
drive_path = ""
kit_ml_dir = "/KIT ML/Fenotypizace - vzchazeni"
if os.path.isdir("/Volumes/GoogleDrive/Shared drives"):
    drive_path = "/Volumes/GoogleDrive/Shared drives"
elif os.path.isdir("G:/Shared drives"):
    drive_path = "G:/Shared drives"
elif not os.path.isdir(drive_path):
    from google.colab import drive
    drive.mount('/content/gdrive')
    drive_path = "/content/gdrive/Shareddrives"

import os
HOME = os.getcwd()
print(HOME)



from IPython.display import display, Image
from PIL import Image
import shutil
import glob
import io
!pip install keras-tcn
import numpy as np
import pandas as pd
import csv
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential
from tcn import TCN
import keras.backend as k_backend
from keras import Input, Model
import os
import csv

Mounted at /content/gdrive
/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.2 MB/s eta 0:00:00


In [2]:
temp_path = drive_path + "/KIT ML 2/Code/Develop/5. Evaluating results/numpy_temp"
model_save_path = drive_path + "/KIT ML 2/Data/Evaluation/"
task_name = "x"

In [3]:
def delete_and_create(temp_path):
    # Odstranění složky a jejího obsahu
    try:
        shutil.rmtree(temp_path, ignore_errors=True)
    except FileNotFoundError:
        pass
    if not os.path.exists(temp_path):
        # Vytvoření složky, pokud neexistuje
        os.makedirs(temp_path)

def import_npy(data_path,names_path, model_path,annotations_path):
    import csv



    new_data = np.load(data_path)

    loaded_model = keras.models.load_model(model_path)
    predictions = loaded_model.predict(new_data)

    predictions_flat = predictions.flatten()
    predictions_list = [[val] for val in predictions_flat]




    predictions_csv =temp_path + "/predictions_temp.csv"
    header = ["guess"]  # Zde nahraďte názvy sloupců podle potřeby
    with open(predictions_csv, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)  # Záhlaví zapsáno jako první řádek
        writer.writerows(predictions_list)


    names = np.load(names_path)
    df = pd.DataFrame(names)
    df = df.iloc[:, 0]
    csv_names_temp = temp_path+"/names_temp.csv"
    custom_header=["names"]
    df.to_csv(csv_names_temp, index=False, header=custom_header)

    names = np.load(annotations_path)
    df = pd.DataFrame(names)
    df = df.iloc[:, 0]
    csv_annot_temp = temp_path+"/annot_temp.csv"
    custom_header=["annot"]
    df.to_csv(csv_annot_temp, index=False, header=custom_header)

    df1 = pd.read_csv(csv_names_temp, header=0)
    df2 = pd.read_csv(csv_annot_temp, header=0)
    df3 = pd.read_csv(predictions_csv, header=0)

    # Combine the two DataFrames side by side
    df_combined = pd.concat([df1, df2, df3], axis=1)

    csv_combined = temp_path + "/BASE.csv"
    df_combined.to_csv(csv_combined, index=False)

def conv_prob_binary():
    input_file_path = temp_path + '/BASE.csv'
    output_file_path = temp_path + '/BASE_conv.csv'

    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

    # Write the header to the output file
        header = next(csv_reader)
        header.append('prob')  # Add the name of the new column
        csv_writer.writerow(header)

        threshHold = 0.5
        for row in csv_reader:
            probability = float(row[2])

            if probability <= threshHold:
                new_column_value = 0
            else:
                new_column_value = 1

            row.append(new_column_value)
            csv_writer.writerow(row)

    os.replace(output_file_path, input_file_path)

    # Načtěte soubor CSV do DataFrame
    df = pd.read_csv(input_file_path)

    # Odeberte sloupec "guess" ze DataFrame
    if 'guess' in df.columns:
        df.drop(columns=['guess'], inplace=True)

    # Uložte upravený DataFrame zpět do souboru CSV
    df.to_csv(output_file_path, index=False)

    os.replace(output_file_path, input_file_path)

def add_loc():
    input_file_path = temp_path+'/BASE.csv'
    output_file_path = temp_path+'/BASE_temp.csv'

    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        header.append("loc")  # Add the name of the new column
        csv_writer.writerow(header)

        threshHold = 0.5
        for row in csv_reader:
            name= row[0]
            shards = name.split("_")
            half = shards[0] + "-" + shards[-1]
            half = half.replace(".png", "")

            row.append(half)
            csv_writer.writerow(row)

    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file_path, input_file_path)

def add_aroud_two():
    input_file_path =temp_path + '/BASE.csv'
    output_file_path = temp_path + '/BASE_temp_+-2.csv'

    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        header.append("prob+-2")  # Add the name of the new column
        csv_writer.writerow(header)

        current_annot = 0
        check_annot=0
        check_name ="x"
        current_name="x"
        half = "x"
        count = 0
        for row in csv_reader:
            if count <= 0:
                half = row[2]
                current_annot = int(row[1])
                current_name = row[3]
                if current_annot != check_annot:
                    if current_name == check_name:
                        half = row[1]
                        count = 2

                check_name = row[3]
                check_annot = int(row[1])

                row.append(half)
                csv_writer.writerow(row)
            else:
                half = row[1]
                row.append(half)
                csv_writer.writerow(row)
                count = count - 1


    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file_path, input_file_path)

def reverse():
    input_file = temp_path + '/BASE.csv'
    output_file = temp_path + '/BASE_temp_rev.csv'
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        headers = next(reader)  # Přečtěte hlavičky

    # Načtěte všechny zbylé řádky ze vstupního CSV souboru do seznamu
    rows = []
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        next(reader)  # Přeskočte hlavičky
        for row in reader:
            rows.append(row)

    # Otočte pořadí řádků v seznamu
    reversed_rows = list(reversed(rows))

    # Zapište hlavičky a obrácené řádky do výstupního CSV souboru
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(headers)  # Zapište hlavičky
        for reversed_row in reversed_rows:
            writer.writerow(reversed_row)

    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file, input_file)

def cleaning_part1():
    input_file_path = temp_path +'/BASE.csv'
    output_file_path = temp_path+'/BASE_temp_+-2.csv'

    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        csv_writer.writerow(header)

        current_annot = 1
        check_annot=1
        check_name ="x"
        current_name="x"
        count = 1
        half = 1
        for row in csv_reader:
            if count <= 0:
                half = row[2]
                current_annot = int(row[1])
                current_name = row[3]
                if current_annot != check_annot:
                    if current_name == check_name:
                        half = row[1]
                        count = 1
                        row[4]= half

                check_name = row[3]
                check_annot = int(row[1])

                csv_writer.writerow(row)
            else:
                half = row[1]
                row[4]= half
                csv_writer.writerow(row)
                count = count - 1


    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file_path, input_file_path)

def cleaning_part2():
    # Načtení existujícího CSV souboru
    input_file_path = temp_path +'/BASE.csv'
    df = pd.read_csv(input_file_path)

    # Vytvoření nových sloupců s různými názvy
    df["above1"] = ""
    df["above2"] = ""
    df["above3"] = ""
    df["below1"] = ""
    df["below2"] = ""
    df["below3"] = ""


    # Otevření nového CSV souboru pro úpravu
    output_file_path = temp_path + '/BASE_temp_cleaning.csv'
    df.to_csv(output_file_path, index=False)

    # Otevření nového CSV souboru pro čtení a úpravu
    df = pd.read_csv(output_file_path)

    # Nastavení hodnot ve sloupci "above" na předchozí hodnoty ze sloupce "predictions" jako celá čísla
    df["above1"] = df["prob"].shift(1).fillna(0).astype(int)
    df["above2"] = df["prob"].shift(2).fillna(0).astype(int)
    df["above3"] = df["prob"].shift(3).fillna(0).astype(int)


    # Nastavení hodnot ve sloupci "below" na následující hodnoty ze sloupce "predictions" jako celá čísla
    df["below1"] = df["prob"].shift(-1).fillna(0).astype(int)
    df["below2"] = df["prob"].shift(-2).fillna(0).astype(int)
    df["below3"] = df["prob"].shift(-3).fillna(0).astype(int)


    # Uložení DataFrame s aktualizovanými sloupci "above" a "below" zpět do souboru
    df.to_csv(output_file_path, index=False)

def reverse_2():
    input_file =temp_path + '/BASE_temp_cleaning.csv'
    output_file = temp_path + '/BASE_temp_rev.csv'
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        headers = next(reader)  # Přečtěte hlavičky

    # Načtěte všechny zbylé řádky ze vstupního CSV souboru do seznamu
    rows = []
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        next(reader)  # Přeskočte hlavičky
        for row in reader:
            rows.append(row)

    # Otočte pořadí řádků v seznamu
    reversed_rows = list(reversed(rows))

    # Zapište hlavičky a obrácené řádky do výstupního CSV souboru
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(headers)  # Zapište hlavičky
        for reversed_row in reversed_rows:
            writer.writerow(reversed_row)

    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file, input_file)


def cleaning_part3():
    input_file_path = temp_path+ '/BASE_temp_cleaning.csv'
    output_file_path = temp_path+'/cBASE.csv'

    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        header.append("cprob")
        csv_writer.writerow(header)

        above1=5
        above2=5
        above3=5
        below1=5
        below2=5
        below3=5
        loc="x"
        prevloc="x"
        keyloc="x"
        con1=False
        annie=False
        for row in csv_reader:
            loc = row[3]
            if loc != prevloc: #kontrola, zda se nezmenil box, pokud ano, tak funkce na nulování se zastaví
                con1=False
                annie=False
            prevloc = loc
            if (row[2]=="1") and (row[5]=="1") and (row[6]=="1") and (row[7]=="1"):
                con1=True
            if (row[2]=="0") and (row[8]=="0") and (row[9]=="0") and (row[10]=="0") and (con1==True):
                annie=True
            if (annie==True) and (row[1]=="0"):
                app = 0
                row.append(app)
            else:
                app=row[2]
                row.append(app)
            csv_writer.writerow(row)



    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    #os.replace(output_file_path, input_file_path)

def reverse3():
    input_file = temp_path + '/cBASE.csv'
    output_file = temp_path + '/cBASE_temp.csv'
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        headers = next(reader)  # Přečtěte hlavičky

    # Načtěte všechny zbylé řádky ze vstupního CSV souboru do seznamu
    rows = []
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        next(reader)  # Přeskočte hlavičky
        for row in reader:
            rows.append(row)

    # Otočte pořadí řádků v seznamu
    reversed_rows = list(reversed(rows))

    # Zapište hlavičky a obrácené řádky do výstupního CSV souboru
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(headers)  # Zapište hlavičky
        for reversed_row in reversed_rows:
            writer.writerow(reversed_row)

    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file, input_file)

def cleaning_part4():
    input_file_path = temp_path + '/cBASE.csv'
    output_file_path = temp_path + '/cBASE_temp_cleaning.csv'

    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        #header.append("cprob")
        csv_writer.writerow(header)

        above1=5
        above2=5
        above3=5
        below1=5
        below2=5
        below3=5
        loc="x"
        prevloc="x"
        keyloc="x"
        con1=False
        annie=False
        for row in csv_reader:
            loc = row[3]
            if loc != prevloc: #kontrola, zda se nezmenil box, pokud ano, tak funkce na nulování se zastaví
                con1=False
                annie=False
            prevloc = loc
            if (row[2]=="0") and (row[5]=="0") and (row[6]=="0") and (row[7]=="0"):
                con1=True
            if (row[2]=="1") and (row[8]=="1") and (row[9]=="1") and (row[10]=="1") and (con1==True):
                annie=True
            if (annie==True) and (row[1]=="1"):
                #app = 1
                row[11]=1
                #row.append(app) #tady jen změnit, aby se to zapsalo do stavajciho vzorce a vymazat else a mělo by to byt ok
            #else:
            #   app=row[2]
              #  row.append("x")
            csv_writer.writerow(row)



    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file_path, input_file_path)

    # Načtěte soubor CSV do DataFrame
    df = pd.read_csv(input_file_path)

    # Odeberte sloupce "guess", "below1" a "below2" ze DataFrame
    columns_to_drop = ['below1', 'below2',"below3","above1","above2","above3"]
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]  # Kontrola existence sloupců v DataFrame
    df.drop(columns=columns_to_drop, inplace=True)

    # Uložte upravený DataFrame zpět do souboru CSV
    df.to_csv(output_file_path, index=False)

    # Pokud chcete nahradit původní soubor upraveným souborem, použijte následující řádek:
    os.replace(output_file_path, input_file_path)

    # Načtěte soubor CSV do DataFrame
    df = pd.read_csv(input_file_path)

    # Změňte pořadí sloupců
    desired_columns_order = ['names', 'loc', 'annot',"prob","prob+-2","cprob"]  # Zde uveďte požadované pořadí sloupců
    df = df.reindex(columns=desired_columns_order)

    # Uložte upravený DataFrame zpět do souboru CSV
    df.to_csv(output_file_path, index=False)


    os.replace(output_file_path, input_file_path)


def eva1(model_path):
    input_file = temp_path + '/cBASE.csv'
    output_file = temp_path + '/cBASE_baseAcc_count.csv'
    with open(input_file, 'r') as input, open(output_file, 'w', newline='') as output:
        reader = csv.reader(input)
        writer = csv.writer(output)

        # Write the header to the output file
        header = next(reader)
        writer.writerow(header)
        truth = 0
        guess = 0
        anomaly = 0
        TRUE = 0
        FALSE = 0

        TP = 0
        TN = 0
        FP = 0
        FN = 0


        # Process the data
        for row in reader:
            truth = int(row[2])
            guess = int(row[3])
            #anom = int(row[3])
            if (truth==guess):
              TRUE +=1
            else:
              FALSE +=1

            if (truth == 1) and (guess == 1):
              TP += 1
            if (truth ==0) and (guess == 0):
              TN += 1
            if (truth == 0) and (guess == 1):
              FP += 1
            if (truth == 1) and (guess == 0):
              FN += 1



            writer.writerow(row)
    task_name = "Accuracy ground"
    df = evaluation_numb(TP, TN, FP, FN, TRUE, FALSE,task=task_name)
    save_into_excel(model_path, model_save_path,sheet_name = task_name,df=df)

def eva2(model_path):
    input_file = temp_path + '/cBASE.csv'
    output_file = temp_path + '/cBASE_Acc+-2_count.csv'
    with open(input_file, 'r') as input, open(output_file, 'w', newline='') as output:
        reader = csv.reader(input)
        writer = csv.writer(output)

        # Write the header to the output file
        header = next(reader)
        writer.writerow(header)
        truth = 0
        guess = 0
        anomaly = 0
        TRUE = 0
        FALSE = 0

        TP = 0
        TN = 0
        FP = 0
        FN = 0


        # Process the data
        for row in reader:
            truth = int(row[2])
            guess = int(row[4])
            #anom = int(row[3])
            if (truth==guess):
              TRUE +=1
            else:
              FALSE +=1

            if (truth == 1) and (guess == 1):
              TP += 1
            if (truth ==0) and (guess == 0):
              TN += 1
            if (truth == 0) and (guess == 1):
              FP += 1
            if (truth == 1) and (guess == 0):
              FN += 1



            writer.writerow(row)
    task_name = "Accuracy +-2"
    df = evaluation_numb(TP, TN, FP, FN, TRUE, FALSE,task=task_name)
    model_path=model_path
    save_into_excel(model_path, model_save_path,sheet_name = task_name,df=df)


def eva3(model_path):
    input_file = temp_path + '/cBASE.csv'
    output_file = temp_path + '/cBASE_Acc_clean_count.csv'
    with open(input_file, 'r') as input, open(output_file, 'w', newline='') as output:
        reader = csv.reader(input)
        writer = csv.writer(output)

        # Write the header to the output file
        header = next(reader)
        writer.writerow(header)
        truth = 0
        guess = 0
        anomaly = 0
        TRUE = 0
        FALSE = 0

        TP = 0
        TN = 0
        FP = 0
        FN = 0


        # Process the data
        for row in reader:
            truth = int(row[2])
            guess = int(row[5])
            #anom = int(row[3])
            if (truth==guess):
              TRUE +=1
            else:
              FALSE +=1

            if (truth == 1) and (guess == 1):
              TP += 1
            if (truth ==0) and (guess == 0):
              TN += 1
            if (truth == 0) and (guess == 1):
              FP += 1
            if (truth == 1) and (guess == 0):
              FN += 1



            writer.writerow(row)
    task_name = "Accuracy after cleaning"
    df = evaluation_numb(TP, TN, FP, FN, TRUE, FALSE,task=task_name)
    model_path=model_path
    save_into_excel(model_path, model_save_path,sheet_name = task_name,df=df)

def boxcount_1():
    input_file_path = temp_path + '/cBASE.csv'
    output_file_path = temp_path + '/cBASE_temp_+-2_first.csv'
    # Načtěte soubor CSV do DataFrame
    df = pd.read_csv(input_file_path)

    # Změňte pořadí sloupců
    desired_columns_order = ['names', 'annot',"prob","loc","prob+-2","cprob"]  # Zde uveďte požadované pořadí sloupců
    df = df.reindex(columns=desired_columns_order)

    # Uložte upravený DataFrame zpět do souboru CSV
    df.to_csv(output_file_path, index=False)


    os.replace(output_file_path, input_file_path)




    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        header.append("cprob+-2")  # Add the name of the new column
        csv_writer.writerow(header)

        current_annot = 0
        check_annot=0
        check_name ="x"
        current_name="x"
        half = "x"
        count = 0
        for row in csv_reader:
            if count <= 0:
                half = row[5]
                current_annot = int(row[1])
                current_name = row[3]
                if current_annot != check_annot:
                    if current_name == check_name:
                        half = row[1]
                        count = 2

                check_name = row[3]
                check_annot = int(row[1])

                row.append(half)
                csv_writer.writerow(row)
            else:
                half = row[1]
                row.append(half)
                csv_writer.writerow(row)
                count = count - 1


    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    #os.replace(output_file_path, input_file_path)


def reverse4():
    input_file = temp_path + '/cBASE_temp_+-2_first.csv'
    output_file =temp_path +  '/BASE_temp_revss.csv'
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        headers = next(reader)  # Přečtěte hlavičky

    # Načtěte všechny zbylé řádky ze vstupního CSV souboru do seznamu
    rows = []
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        next(reader)  # Přeskočte hlavičky
        for row in reader:
            rows.append(row)

    # Otočte pořadí řádků v seznamu
    reversed_rows = list(reversed(rows))

    # Zapište hlavičky a obrácené řádky do výstupního CSV souboru
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(headers)  # Zapište hlavičky
        for reversed_row in reversed_rows:
            writer.writerow(reversed_row)

    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file, input_file)


def boxcount_2():
    input_file_path = temp_path+'/cBASE_temp_+-2_first.csv'
    output_file_path = temp_path +'/BASE_temp_+-2.csvdfds'

    with open(input_file_path, 'r') as input_file,open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

        # Write the header to the output file
        header = next(csv_reader)
        csv_writer.writerow(header)

        current_annot = 1
        check_annot=1
        check_name ="x"
        current_name="x"
        count = 1
        half = 1
        for row in csv_reader:
            if count <= 0:
                half = row[5]
                current_annot = int(row[1])
                current_name = row[3]
                if current_annot != check_annot:
                    if current_name == check_name:
                        half = row[1]
                        count = 1
                        row[4]= half

                check_name = row[3]
                check_annot = int(row[1])

                csv_writer.writerow(row)
            else:
                half = row[1]
                row[4]= half
                csv_writer.writerow(row)
                count = count - 1


    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file_path, input_file_path)

def reverse5():
    input_file = temp_path + '/cBASE_temp_+-2_first.csv'
    output_file = temp_path + '/BASE_temp_revss.csv'
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        headers = next(reader)  # Přečtěte hlavičky

    # Načtěte všechny zbylé řádky ze vstupního CSV souboru do seznamu
    rows = []
    with open(input_file, 'r') as input:
        reader = csv.reader(input)
        next(reader)  # Přeskočte hlavičky
        for row in reader:
            rows.append(row)

    # Otočte pořadí řádků v seznamu
    reversed_rows = list(reversed(rows))

    # Zapište hlavičky a obrácené řádky do výstupního CSV souboru
    with open(output_file, 'w', newline='') as output:
        writer = csv.writer(output)
        writer.writerow(headers)  # Zapište hlavičky
        for reversed_row in reversed_rows:
            writer.writerow(reversed_row)

    # Po dokončení operací zkopírujeme obsah dočasného souboru na původní soubor
    os.replace(output_file, input_file)

def box_count(model_path):
    input_file_path = temp_path + '/cBASE_temp_+-2_first.csv'
    output_file_path = temp_path + '/cBASE_final+-2.csv'


    with open(input_file_path, 'r') as input_file, open(output_file_path, 'w', newline='') as output_file:
        csv_reader = csv.reader(input_file)
        csv_writer = csv.writer(output_file)

    # Write the header to the output file
        header = next(csv_reader)
        header.append('box')  # Add the name of the new column
        csv_writer.writerow(header)

        checkloc = True
        prevloc= "x"
        currenctloc="x"
        error1 = 0
        error2 = 0
        error3 = 0
        error4 = 0
        box_right1=0
        box_right2=0
        box_right3=0
        box_right4=0
        box_wrong1=0
        box_wrong2=0
        box_wrong3=0
        box_wrong4=0
        for row in csv_reader:
            if (prevloc=="x"):
                prevloc = row[3]

            currenctloc = row[3]

            if (row[1] != row[2]):
                error1 = 1

            if (row[1] != row[4]):
                error2 = 1

            if (row[1] != row[5]):
                error3 = 1

            if (row[1] != row[6]):
                error4 = 1


            if (currenctloc!=prevloc):
                if (error1<1):
                    box_right1 += 1
                    error1 = 0
                else:
                    box_wrong1 += 1
                    error1 =0

            if (currenctloc!=prevloc):
                if (error2<1):
                    box_right2 += 1
                    error2 = 0
                else:
                    box_wrong2 += 1
                    error2 =0


            if (currenctloc!=prevloc):
                if (error3<1):
                    box_right3 += 1
                    error3 = 0
                else:
                    box_wrong3 += 1
                    error3 =0

            if (currenctloc!=prevloc):
                if (error4<1):
                    box_right4 += 1
                    error4 = 0
                else:
                    box_wrong4 += 1
                    error4 =0




            prevloc = row[3]
            new_column_value="x"

            row.append(new_column_value)
            csv_writer.writerow(row)
    data = {
        "Boxes ground:": [box_right1, box_wrong1, box_right1/(box_right1+box_wrong1)],
        "Boxes +-2": [box_right2, box_wrong2,box_right2/(box_right2+box_wrong2)],
        "Boxes after clean": [box_right3, box_wrong3,box_right3/(box_right3+box_wrong3)],
        "Boxes after clean +-2": [box_right4, box_wrong4, box_right4/(box_right4+box_wrong4)]
    }

    # Vytvoření DataFrame
    df = pd.DataFrame(data, index=["True", "False", "%"])
    df = df.T
    task_name = "Boxes"
    print(df)
    model_path=model_path
    save_into_excel(model_path, model_save_path,sheet_name = task_name,df=df)




    #os.replace(output_file_path, input_file_path)

def eva4(model_path):
    input_file = temp_path + '/cBASE_temp_+-2_first.csv'
    output_file = temp_path + '/cBASE_Acc_clean+-2_count.csv'
    with open(input_file, 'r') as input, open(output_file, 'w', newline='') as output:
        reader = csv.reader(input)
        writer = csv.writer(output)

        # Write the header to the output file
        header = next(reader)
        writer.writerow(header)
        truth = 0
        guess = 0
        anomaly = 0
        TRUE = 0
        FALSE = 0

        TP = 0
        TN = 0
        FP = 0
        FN = 0


        # Process the data
        for row in reader:
            truth = int(row[1])
            guess = int(row[6])
            #anom = int(row[3])
            if (truth==guess):
              TRUE +=1
            else:
              FALSE +=1

            if (truth == 1) and (guess == 1):
              TP += 1
            if (truth ==0) and (guess == 0):
              TN += 1
            if (truth == 0) and (guess == 1):
              FP += 1
            if (truth == 1) and (guess == 0):
              FN += 1



            writer.writerow(row)
    task_name = "Accuracy +-2 after cleaning"
    df = evaluation_numb(TP, TN, FP, FN, TRUE, FALSE,task=task_name)
    model_path=model_path
    save_into_excel(model_path, model_save_path,sheet_name = task_name,df=df)


def evaluation_numb(TP, TN, FP, FN, TRUE, FALSE,task=task_name):
    data = {
        "Value": [TP, TN, FP, FN, TRUE, FALSE, TRUE/(TRUE+FALSE)]
    }

    # Vytvoření DataFrame
    df = pd.DataFrame(data, index=["TP", "TN", "FP", "FN", "True", "False", "Accuracy"])

    # Transponování DataFrame
    df = df.T

    # Zobrazení DataFrame
    print(task)
    print(df)
    print("-")
    return df

###################################################################

def save_into_excel(model_path, model_save_path,sheet_name,df):
    model_name=model_path.split("/")[-1]
    eva_path = model_save_path + model_name +".xlsx"
    # Vytvoření prázdného DataFrame
    #df = pd.DataFrame()
    df = df
    # Kontrola existence souboru Excel
    if os.path.isfile(eva_path):
        # Pokud soubor existuje, načtení existujícího souboru
        existing_excel = pd.ExcelFile(eva_path)
        # Přidání nebo aktualizace listu
        with pd.ExcelWriter(eva_path, engine='openpyxl', mode='a') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        # Pokud soubor neexistuje, vytvoření nového souboru a uložení DataFrame
        df.to_excel(eva_path, sheet_name=sheet_name, index=False)

def generate_csv(temp_path,model_path,model_save_path):
    # Načtení CSV souboru
    model_name=model_path.split("/")[-1]
    df = pd.read_csv(temp_path + "/cBASE_Acc_clean+-2_count.csv")

    # Vymazání sloupce podle jména
    del df["loc"]  # Nahraďte "nazev_sloupce" jménem sloupce, který chcete vymazat

    # Uložení změn zpět do CSV souboru
    dest_name = model_save_path + model_name + ".csv"
    df.to_csv(dest_name, index=False)

def evaluation(data_path, model_path, names_path,annotations_path):
    delete_and_create(temp_path)
    import_npy(data_path=data_path, model_path=model_path, names_path=names_path,annotations_path=annotations_path)
    conv_prob_binary()
    add_loc()
    add_aroud_two()
    reverse()
    cleaning_part1()
    reverse()
    cleaning_part2()
    reverse_2()
    cleaning_part3()
    reverse3()
    cleaning_part4()
    eva1(model_path)
    eva2(model_path)
    eva3(model_path)
    boxcount_1()
    reverse4()
    boxcount_2()
    reverse5()
    eva4(model_path)
    box_count(model_path)
    generate_csv(temp_path,model_path,model_save_path)



# EXECUTION:

# ef_tnc_2-8 all data

In [4]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_2_8_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_tuner_2-8_all_data",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_2_8_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_2_8_[43, 46, 48].npy"
            )

462/462 [==============================] - 149s 318ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5566.0  8527.0  346.0  327.0  14093.0  673.0  0.954422
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5650.0  8581.0  292.0  243.0  14231.0  535.0  0.963768
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5748.0  8759.0  114.0  145.0  14507.0  259.0   0.98246
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5828.0  8759.0  114.0  65.0  14587.0  179.0  0.987878
-
                        True  False         %
Boxes ground:           15.0  192.0  0.072464
Boxes +-2               35.0  172.0  0.169082
Boxes after clean       95.0  112.0  0.458937
Boxes after clean +-2  155.0   52.0  0.748792


# ef_lstm_tuner_2-8

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_2_8_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_lstm_tuner_2-8",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_2_8_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_2_8_[43, 46, 48].npy"
            )

462/462 [==============================] - 161s 342ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5644.0  8379.0  494.0  249.0  14023.0  743.0  0.949682
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5714.0  8437.0  436.0  179.0  14151.0  615.0   0.95835
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5742.0  8661.0  212.0  151.0  14403.0  363.0  0.975416
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5809.0  8661.0  212.0  84.0  14470.0  296.0  0.979954
-
                        True  False         %
Boxes ground:           43.0  164.0  0.207729
Boxes +-2               85.0  122.0  0.410628
Boxes after clean      104.0  103.0  0.502415
Boxes after clean +-2  148.0   59.0  0.714976


# ef_tnc_2-8

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_2_8_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_2-8",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_2_8_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_2_8_[43, 46, 48].npy"
            )

462/462 [==============================] - 141s 299ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5544.0  8521.0  352.0  349.0  14065.0  701.0  0.952526
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5606.0  8583.0  290.0  287.0  14189.0  577.0  0.960924
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5745.0  8686.0  187.0  148.0  14431.0  335.0  0.977313
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5805.0  8686.0  187.0  88.0  14491.0  275.0  0.981376
-
                        True  False         %
Boxes ground:            1.0  206.0  0.004831
Boxes +-2                2.0  205.0  0.009662
Boxes after clean      105.0  102.0  0.507246
Boxes after clean +-2  143.0   64.0  0.690821


# ef_tnc_tuner_2-8

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_2_8_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_tuner_2-8",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_2_8_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_2_8_[43, 46, 48].npy"
            )

462/462 [==============================] - 140s 299ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5556.0  8596.0  277.0  337.0  14152.0  614.0  0.958418
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5614.0  8658.0  215.0  279.0  14272.0  494.0  0.966545
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5770.0  8741.0  132.0  123.0  14511.0  255.0  0.982731
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5827.0  8741.0  132.0  66.0  14568.0  198.0  0.986591
-
                        True  False         %
Boxes ground:            1.0  206.0  0.004831
Boxes +-2                4.0  203.0  0.019324
Boxes after clean      106.0  101.0  0.512077
Boxes after clean +-2  143.0   64.0  0.690821


# ef_lstm_tunner_3-9

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_3_9_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_lstm_tunner_3-9",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_3_9_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_3_9_[43, 46, 48].npy"
            )

462/462 [==============================] - 202s 433ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5395.0  8515.0  358.0  497.0  13910.0  855.0  0.942093
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5491.0  8615.0  258.0  401.0  14106.0  659.0  0.955367
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5715.0  8699.0  174.0  177.0  14414.0  351.0  0.976228
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5809.0  8699.0  174.0  83.0  14508.0  257.0  0.982594
-
                        True  False         %
Boxes ground:            1.0  206.0  0.004831
Boxes +-2                7.0  200.0  0.033816
Boxes after clean       66.0  141.0  0.318841
Boxes after clean +-2  121.0   86.0  0.584541


# ef_tnc_3-9

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_3_9_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_3-9",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_3_9_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_3_9_[43, 46, 48].npy"
            )

462/462 [==============================] - 201s 429ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5668.0  8500.0  373.0  224.0  14168.0  597.0  0.959567
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5706.0  8664.0  209.0  186.0  14370.0  395.0  0.973248
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5772.0  8618.0  255.0  120.0  14390.0  375.0  0.974602
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5810.0  8618.0  255.0  82.0  14428.0  337.0  0.977176
-
                       True  False         %
Boxes ground:          23.0  184.0  0.111111
Boxes +-2              81.0  126.0  0.391304
Boxes after clean      54.0  153.0  0.260870
Boxes after clean +-2  73.0  134.0  0.352657


# ef_tnc_tunner_3-9

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_3_9_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_tunner_3-9",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_3_9_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_3_9_[43, 46, 48].npy"
            )

462/462 [==============================] - 205s 438ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5730.0  8599.0  274.0  162.0  14329.0  436.0  0.970471
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5764.0  8765.0  108.0  128.0  14529.0  236.0  0.984016
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5775.0  8684.0  189.0  117.0  14459.0  306.0  0.979275
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5809.0  8684.0  189.0  83.0  14493.0  272.0  0.981578
-
                        True  False         %
Boxes ground:           42.0  165.0  0.202899
Boxes +-2              150.0   57.0  0.724638
Boxes after clean       53.0  154.0  0.256039
Boxes after clean +-2   68.0  139.0  0.328502


# ef_lstm_tunner_4-10

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_4_10_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_lstm_tuner_4-10",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_4_10_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_4_10_[43, 46, 48].npy"
            )

462/462 [==============================] - 303s 648ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5549.0  8519.0  354.0  342.0  14068.0  696.0  0.952858
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5598.0  8752.0  121.0  293.0  14350.0  414.0  0.971959
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5760.0  8606.0  267.0  131.0  14366.0  398.0  0.973043
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5809.0  8606.0  267.0  82.0  14415.0  349.0  0.976361
-
                       True  False         %
Boxes ground:           7.0  200.0  0.033816
Boxes +-2              25.0  182.0  0.120773
Boxes after clean      35.0  172.0  0.169082
Boxes after clean +-2  55.0  152.0  0.265700


# ef_tnc_4-10

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_4_10_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_4-10",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_4_10_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_4_10_[43, 46, 48].npy"
            )

462/462 [==============================] - 259s 556ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5575.0  8532.0  341.0  316.0  14107.0  657.0    0.9555
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5600.0  8748.0  125.0  291.0  14348.0  416.0  0.971823
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5781.0  8619.0  254.0  110.0  14400.0  364.0  0.975345
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5806.0  8619.0  254.0  85.0  14425.0  339.0  0.977039
-
                       True  False         %
Boxes ground:           0.0  207.0  0.000000
Boxes +-2               2.0  205.0  0.009662
Boxes after clean      39.0  168.0  0.188406
Boxes after clean +-2  46.0  161.0  0.222222


# ef_tnc_tuner_4-10

In [ ]:
evaluation(data_path=drive_path + "/KIT ML 2/Data/numpy/cropped64/data_64_4_10_[43, 46, 48].npy",
            model_path=drive_path + "/KIT ML 2/Data/Models/ef_tnc_tuner_4-10",
            names_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/image_names_64_4_10_[43, 46, 48].npy",
            annotations_path=drive_path+ "/KIT ML 2/Data/numpy/cropped64/annotations_64_4_10_[43, 46, 48].npy"
            )

462/462 [==============================] - 262s 560ms/step
Accuracy ground
           TP      TN     FP     FN     True  False  Accuracy
Value  5417.0  8525.0  348.0  474.0  13942.0  822.0  0.944324
-
Accuracy +-2
           TP      TN     FP     FN     True  False  Accuracy
Value  5440.0  8766.0  107.0  451.0  14206.0  558.0  0.962205
-
Accuracy after cleaning
           TP      TN     FP     FN     True  False  Accuracy
Value  5784.0  8599.0  274.0  107.0  14383.0  381.0  0.974194
-
Accuracy +-2 after cleaning
           TP      TN     FP    FN     True  False  Accuracy
Value  5807.0  8599.0  274.0  84.0  14406.0  358.0  0.975752
-
                       True  False         %
Boxes ground:           0.0  207.0  0.000000
Boxes +-2               0.0  207.0  0.000000
Boxes after clean      27.0  180.0  0.130435
Boxes after clean +-2  32.0  175.0  0.154589
